In [61]:
import random
import requests
from bs4 import BeautifulSoup
import time
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [62]:
def get_proxies():    
    url = 'https://free-proxy-list.net/anonymous-proxy.html'
    headers = {"Accept-Language": "en-US,en;q=0.5"}
    response = requests.get(url, headers=headers)
    content = BeautifulSoup(response.content, 'html.parser')
    rows = content.find('table').find_all('tr')
    rows = rows[1:]
    proxies = []
    for row in rows:
        tds = row.find_all('td')
        if tds[-2].text == 'no':
            proxy = f'http://{tds[0].text}:{tds[1].text}'
        if tds[-2].text == 'yes':
            proxy = f'https://{tds[0].text}:{tds[1].text}'
        proxies.append(proxy)
    return proxies

In [63]:
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)

ip_addresses = get_proxies()
print(ip_addresses)

def proxy_request(url, ip_addresses):
    session = requests.Session()
    session.mount('https://', adapter)

    while True:
        try:
            ua = UserAgent()    
            headers = {"Accept-Language": "en-US,en;q=0.5", 'User-Agent': ua.random}
            proxy = random.randint(0, len(ip_addresses) - 1)
            proxies = {"https": ip_addresses[proxy]}
            response = session.get(url, proxies=proxies, headers=headers)
            if response.status_code == 200:
                break
        except Exception as e:
            print(f"Error: {e}")

    return response

ConnectionError: HTTPSConnectionPool(host='free-proxy-list.net', port=443): Max retries exceeded with url: /anonymous-proxy.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10e8af430>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [64]:
url = 'https://www.gsmarena.com/makers.php3'
response = proxy_request(url, ip_addresses)
content = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

KeyboardInterrupt: 

In [ ]:
prefix = 'https://www.gsmarena.com/makers.php3/'
maker_urls = dict()
query = list(map(lambda x: x.find('a'), content.find('table').find_all('td')))
maker_urls = dict(zip(list(map(lambda x: x.text.split()[0], query)), list(map(lambda x: prefix + x.attrs['href'], query))))

In [ ]:
maker_urls

{'Acer100': 'https://www.gsmarena.com/makers.php3/acer-phones-59.php',
 'alcatel409': 'https://www.gsmarena.com/makers.php3/alcatel-phones-5.php',
 'Allview157': 'https://www.gsmarena.com/makers.php3/allview-phones-88.php',
 'Amazon25': 'https://www.gsmarena.com/makers.php3/amazon-phones-76.php',
 'Amoi47': 'https://www.gsmarena.com/makers.php3/amoi-phones-28.php',
 'Apple118': 'https://www.gsmarena.com/makers.php3/apple-phones-48.php',
 'Archos43': 'https://www.gsmarena.com/makers.php3/archos-phones-90.php',
 'Asus203': 'https://www.gsmarena.com/makers.php3/asus-phones-46.php',
 'AT&T4': 'https://www.gsmarena.com/makers.php3/at&t-phones-57.php',
 'Benefon9': 'https://www.gsmarena.com/makers.php3/benefon-phones-15.php',
 'BenQ35': 'https://www.gsmarena.com/makers.php3/benq-phones-31.php',
 'BenQ-Siemens28': 'https://www.gsmarena.com/makers.php3/benq_siemens-phones-42.php',
 'Bird61': 'https://www.gsmarena.com/makers.php3/bird-phones-34.php',
 'BlackBerry92': 'https://www.gsmarena.com/m

In [ ]:
maker_urls['Acer100']

'https://www.gsmarena.com/makers.php3/acer-phones-59.php'

In [ ]:
prefix = 'https://www.gsmarena.com/'
for maker in maker_urls:
    response = proxy_request(maker_urls[maker], ip_addresses)
    content = BeautifulSoup(response.content, 'html.parser')
    maker_urls[maker] = [maker_urls[maker]]
    while True:
        try:
            query = content.find('div', {'class': 'nav-pages'}).find('a', {'title': 'Next page'})
            response = proxy_request(prefix + query.attrs['href'], ip_addresses)
        except:
            print(maker_urls[maker])
            break

['https://www.gsmarena.com/makers.php3/acer-phones-59.php']


KeyboardInterrupt: 

In [ ]:
maker_urls

In [ ]:
prefix = 'https://www.gsmarena.com/'
for maker in maker_urls:
    response = proxy_request(maker_urls[maker], ip_addresses)
    content = BeautifulSoup(response.content, 'html.parser')
    query = content.find('div', {'class': 'makers'}).find_all('a')
    maker_urls[maker] = dict(map(lambda x: (x.text, prefix + x.attrs['href']), query))
    print(maker_urls[maker])

{'Chromebook Tab 10': 'https://www.gsmarena.com/acer_chromebook_tab_10-9139.php', 'Iconia Talk S': 'https://www.gsmarena.com/acer_iconia_talk_s-8306.php', 'Liquid Z6 Plus': 'https://www.gsmarena.com/acer_liquid_z6_plus-8305.php', 'Liquid Z6': 'https://www.gsmarena.com/acer_liquid_z6-8304.php', 'Iconia Tab 10 A3-A40': 'https://www.gsmarena.com/acer_iconia_tab_10_a3_a40-8080.php', 'Liquid X2': 'https://www.gsmarena.com/acer_liquid_x2-8034.php', 'Liquid Jade 2': 'https://www.gsmarena.com/acer_liquid_jade_2-7956.php', 'Liquid Zest Plus': 'https://www.gsmarena.com/acer_liquid_zest_plus-8059.php', 'Liquid Zest': 'https://www.gsmarena.com/acer_liquid_zest-7955.php', 'Predator 8': 'https://www.gsmarena.com/acer_predator_8-7750.php', 'Liquid Jade Primo': 'https://www.gsmarena.com/acer_liquid_jade_primo-7650.php', 'Liquid Z330': 'https://www.gsmarena.com/acer_liquid_z330-7530.php', 'Liquid Z320': 'https://www.gsmarena.com/acer_liquid_z320-7531.php', 'Liquid Z630S': 'https://www.gsmarena.com/acer

In [ ]:
maker_urls.keys()

dict_keys(['Acer100', 'alcatel409', 'Allview157', 'Amazon25', 'Amoi47', 'Apple118', 'Archos43', 'Asus203', 'AT&T4', 'Benefon9', 'BenQ35', 'BenQ-Siemens28', 'Bird61', 'BlackBerry92', 'Blackview89', 'BLU369', 'Bosch10', 'BQ20', 'Casio5', 'Cat22', 'Celkon229', 'Chea12', 'Coolpad46', 'Cubot57', 'Dell20', 'Doogee66', 'Emporia15', 'Energizer69', 'Ericsson40', 'Eten22', 'Fairphone4', 'Fujitsu', 'Garmin-Asus5', 'Gigabyte63', 'Gionee95', 'Google27', 'Haier59', 'Honor206', 'HP41', 'HTC287', 'Huawei441', 'i-mate34', 'i-mobile37', 'Icemobile61', 'Infinix125', 'Innostream18', 'iNQ5', 'Intex15', 'itel32', 'Jolla3', 'Karbonn60', 'Kyocera24', 'Lava145', 'LeEco9', 'Lenovo246', 'LG667', 'Maxon31', 'Maxwest41', 'Meizu73', 'Micromax289', 'Microsoft32', 'Mitac12', 'Mitsubishi25', 'Modu8', 'Motorola609', 'MWg5', 'NEC73', 'Neonode3', 'NIU30', 'Nokia576', 'Nothing3', 'Nvidia3', 'O245', 'OnePlus66', 'Oppo295', 'Orange19', 'Oukitel32', 'Palm17', 'Panasonic123', 'Pantech72', 'Parla10', 'Philips229', 'Plum113', '

In [ ]:
brand_list = [
    'Samsung1382',
    'Apple118',
    'Xiaomi377',
    'Huawei441',
    'alcatel409',
    'Sony',
    'Sony158',
    'LG667',
    'Lenovo246',
    'ZTE369',
    'Nokia576',
    'BLU369',
    'Infinix125',
    'HTC287',
    'Asus203',
]

In [ ]:
info_dict_with_headers = {}

for brand in maker_urls:
    if brand in brand_list:
        info_dict_with_headers[brand] = {}
        for device in maker_urls[brand]:
            print(device)
            url = maker_urls[brand][device]
            url = url.replace('https://www.gsmarena.com/makers.php3', 'https://www.gsmarena.com/makers.php3/')
            time.sleep(3)
            response = proxy_request(url, ip_addresses)
            soup = BeautifulSoup(response.content, 'html.parser')
            table_list = soup.find_all('table')
            info_dict_with_headers[brand][device] = {}

            for table in table_list:
                # Attempt to find the table header
                header = table.find('th')
                if header:
                    header_text = header.get_text(strip=True)
                else:
                    # If no <th> is found, or there is a need to identify the header differently, adjust here
                    header_text = "Unknown"  # Placeholder if no header is found or defined differently

                # Initialize the dictionary for this header if not already present
                if header_text not in info_dict_with_headers:
                    info_dict_with_headers[brand][device][header_text] = []

                last_key = None  # Keep track of the last non-empty key for rows without a 'ttl'

                # Iterate through each row in the table
                for row in table.find_all('tr'):
                    ttl_element = row.find("td", class_="ttl")
                    nfo_element = row.find("td", class_="nfo")

                    if ttl_element and ttl_element.get_text(strip=True):
                        # If ttl_element has text, update last_key
                        key = ttl_element.get_text(strip=True)
                        last_key = key
                    else:
                        # If ttl_element is empty, use the last_key
                        key = last_key

                    if nfo_element:
                        value = nfo_element.get_text(strip=True)
                        # Append the key-value pair under the current header
                        if key:
                            info_dict_with_headers[brand][device][header_text].append({key: value})
                        else:
                            # Handle case where there's a value without a key (could append to last key's value or handle as needed)
                            pass

            print(f'{device} Done!')
        print(f'{brand} Done!')
    else:
        continue

1B (2022)
1B (2022) Done!
1L Pro (2021)
1L Pro (2021) Done!
1 (2021)
1 (2021) Done!
3L (2021)
3L (2021) Done!
1S (2021)
1S (2021) Done!
1L (2021)
1L (2021) Done!
1V (2021)
1V (2021) Done!
1SE (2020)
1SE (2020) Done!
Go Flip 4
Go Flip 4 Done!
Go Flip 3
Go Flip 3 Done!
Go Flip V
Go Flip V Done!
3T10 2020
3T10 2020 Done!
3X (2020)
3X (2020) Done!
3L (2020)
3L (2020) Done!
1S (2020)
1S (2020) Done!
1V (2020)
1V (2020) Done!
1B (2020)
1B (2020) Done!
3v (2019)
3v (2019) Done!
3x (2019)
3x (2019) Done!
1v (2019)
1v (2019) Done!
3T 8
3T 8 Done!
Smart Tab 7
Smart Tab 7 Done!
3T 10
3T 10 Done!
3L
3L Done!
3 (2019)
3 (2019) Done!
1s
1s Done!
1x (2019)
1x (2019) Done!
1c (2019)
1c (2019) Done!
1
1 Done!
Tetra
Tetra Done!
7
7 Done!
5v
5v Done!
5
5 Done!
3v
3v Done!
3x (2018)
3x (2018) Done!
3
3 Done!
1x
1x Done!
3c
3c Done!
3088
3088 Done!
1T 10
1T 10 Done!
1T 7
1T 7 Done!
Idol 5s
Idol 5s Done!
Idol 5
Idol 5 Done!
A7 XL
A7 XL Done!
A7
A7 Done!
U5 HD
U5 HD Done!
Pulsemix
Pulsemix Done!
Idol 5s (USA

In [ ]:
info_dict_with_headers['ZTE369']

{'nubia Z60 Ultra': {'Network': [{'Technology': 'GSM / CDMA / HSPA / CDMA2000 / LTE / 5G'},
   {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
   {'2G bands': 'CDMA 800'},
   {'3G bands': 'HSDPA 850 / 900 / 1700(AWS) / 1900 / 2100'},
   {'3G bands': 'CDMA2000 1x'},
   {'4G bands': '1, 2, 3, 4, 5, 7, 8, 12, 17, 18, 19, 20, 26, 28, 34, 38, 39, 40, 41, 66 - International'},
   {'4G bands': '1, 2, 3, 4, 5, 7, 8, 12, 17, 18, 19, 20, 26, 28, 34, 38, 39, 40, 41 - China'},
   {'5G bands': '1, 2, 3, 5, 7, 8, 28, 38, 40, 41, 66, 77, 78 SA/NSA - International'},
   {'5G bands': '1, 3, 5, 8, 28, 41, 78 SA/NSA - China'},
   {'Speed': 'HSPA, LTE-A (CA), 5G'}],
  'Launch': [{'Announced': '2023, December 19'},
   {'Status': 'Available. Released 2023, December 29'}],
  'Body': [{'Dimensions': '164 x 76.4 x 8.8 mm (6.46 x 3.01 x 0.35 in)'},
   {'Weight': '246 g (8.68 oz)'},
   {'SIM': 'Dual SIM (Nano-SIM, dual stand-by)'},
   {'SIM': 'IP68 dust/water resistant (up to 1.5m for 30 min)'}],
  

In [ ]:
info_dict_with_headers = {}

for brand in maker_urls:
    if brand not in brand_list:
        info_dict_with_headers[brand] = {}
        for device in maker_urls[brand]:
            print(device)
            url = maker_urls[brand][device]
            url = url.replace('https://www.gsmarena.com/makers.php3', 'https://www.gsmarena.com/makers.php3/')
            time.sleep(3)
            response = proxy_request(url, ip_addresses)
            soup = BeautifulSoup(response.content, 'html.parser')
            table_list = soup.find_all('table')
            info_dict_with_headers[brand][device] = {}

            for table in table_list:
                # Attempt to find the table header
                header = table.find('th')
                if header:
                    header_text = header.get_text(strip=True)
                else:
                    # If no <th> is found, or there is a need to identify the header differently, adjust here
                    header_text = "Unknown"  # Placeholder if no header is found or defined differently

                # Initialize the dictionary for this header if not already present
                if header_text not in info_dict_with_headers:
                    info_dict_with_headers[brand][device][header_text] = []

                last_key = None  # Keep track of the last non-empty key for rows without a 'ttl'

                # Iterate through each row in the table
                for row in table.find_all('tr'):
                    ttl_element = row.find("td", class_="ttl")
                    nfo_element = row.find("td", class_="nfo")

                    if ttl_element and ttl_element.get_text(strip=True):
                        # If ttl_element has text, update last_key
                        key = ttl_element.get_text(strip=True)
                        last_key = key
                    else:
                        # If ttl_element is empty, use the last_key
                        key = last_key

                    if nfo_element:
                        value = nfo_element.get_text(strip=True)
                        # Append the key-value pair under the current header
                        if key:
                            info_dict_with_headers[brand][device][header_text].append({key: value})
                        else:
                            # Handle case where there's a value without a key (could append to last key's value or handle as needed)
                            pass

            print(f'{device} Done!')
        print(f'{brand} Done!')
    else:
        continue

Chromebook Tab 10
Chromebook Tab 10 Done!
Iconia Talk S
Iconia Talk S Done!
Liquid Z6 Plus
Liquid Z6 Plus Done!
Liquid Z6
Liquid Z6 Done!
Iconia Tab 10 A3-A40
Iconia Tab 10 A3-A40 Done!
Liquid X2
Liquid X2 Done!
Liquid Jade 2
Liquid Jade 2 Done!
Liquid Zest Plus
Liquid Zest Plus Done!
Liquid Zest
Liquid Zest Done!
Predator 8
Predator 8 Done!
Liquid Jade Primo
Liquid Jade Primo Done!
Liquid Z330
Liquid Z330 Done!
Liquid Z320
Liquid Z320 Done!
Liquid Z630S
Liquid Z630S Done!
Liquid Z630
Liquid Z630 Done!
Liquid Z530S
Liquid Z530S Done!
Liquid Z530
Liquid Z530 Done!
Liquid M330
Liquid M330 Done!
Liquid M320
Liquid M320 Done!
Iconia Tab 10 A3-A30
Iconia Tab 10 A3-A30 Done!
Iconia One 8 B1-820
Iconia One 8 B1-820 Done!
Iconia Tab A3-A20
Iconia Tab A3-A20 Done!
Iconia Tab A3-A20FHD
Iconia Tab A3-A20FHD Done!
Liquid Jade Z
Liquid Jade Z Done!
Liquid Z520
Liquid Z520 Done!
Liquid Z220
Liquid Z220 Done!
Liquid M220
Liquid M220 Done!
Liquid Z410
Liquid Z410 Done!
Liquid Jade S
Liquid Jade S Done

In [ ]:
import json

In [ ]:
file_path = 'data.json'
with open(file_path, 'w') as json_file:
    json.dump(info_dict_with_headers, json_file)

In [ ]:
info_dict_with_headers['YU13']

{'Ace': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
   {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
   {'3G bands': 'HSDPA 900 / 2100'},
   {'4G bands': '1, 3, 5, 40, 41'},
   {'Speed': 'HSPA 21.1/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
  'Launch': [{'Announced': '2018, August'},
   {'Status': 'Available. Released 2018, September'}],
  'Body': [{'Dimensions': '148 x 70.5 x 9.6 mm (5.83 x 2.78 x 0.38 in)'},
   {'Weight': '159 g (5.61 oz)'},
   {'SIM': 'Dual SIM (Nano-SIM, dual stand-by)'}],
  'Display': [{'Type': 'IPS LCD'},
   {'Size': '5.45 inches, 76.7 cm2(~73.5% screen-to-body ratio)'},
   {'Resolution': '720 x 1440 pixels, 18:9 ratio (~295 ppi density)'}],
  'Platform': [{'OS': 'Android 8.0 (Oreo)'},
   {'Chipset': 'Mediatek MT6739WW (28 nm)'},
   {'CPU': 'Quad-core 1.5 GHz Cortex-A53'},
   {'GPU': 'PowerVR GE8100'}],
  'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
   {'Internal': '16GB 2GB RAM, 32GB 3GB RAM'},
   {'Internal': 'eMMC 5.1'}],
  'Main Camera